# Data Cleaning Dataset de Inegi

En este notebook procederemos con la limpieza del dataset de Inegi

In [1]:
import pandas as pd
import seaborn as sns
from thefuzz import fuzz
from thefuzz import process
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import re
import unidecode

In [2]:
def clean_text(text):
    """
    De esta manera tenemos el texto sin espacios blancos extra y sobre todo con todas las palabras con capitalización correcta.
    """
    if pd.isna(text):
        return text
    text = text.strip()  # Eliminate white spaces
    text = text.lower()  # Convert to lowercase
    text = unidecode.unidecode(text)  # Remove accents
    text = re.sub('-.*-', '', text)
    text = re.sub('\s+', ' ', text)  # Eliminate extra white spaces
    text = re.sub('^\s+|\s+?$', '', text)  # Eliminate spaces at the beginning and end
    return text

In [3]:
path_dataset_inegi = '../../data/dataset_inegi.csv'
dataset_inegi = pd.read_csv(path_dataset_inegi, encoding='cp1252', dtype={'CVE_ENT': str, 'CVE_MUN': str})

In [4]:
# Revisamos las columnas del dataset
dataset_inegi.columns

Index(['MAPA', 'Estatus', 'CVE_ENT', 'NOM_ENT', 'NOM_ABR', 'CVE_MUN',
       'NOM_MUN', 'CVE_LOC', 'NOM_LOC', 'AMBITO', 'LATITUD', 'LONGITUD',
       'LAT_DECIMAL', 'LON_DECIMAL', 'ALTITUD', 'CVE_CARTA', 'POB_TOTAL',
       'POB_MASCULINA', 'POB_FEMENINA', 'TOTAL DE VIVIENDAS HABITADAS'],
      dtype='object')

In [5]:
dataset_inegi.head()

,MAPA,Estatus,CVE_ENT,NOM_ENT,NOM_ABR,CVE_MUN,NOM_MUN,CVE_LOC,NOM_LOC,AMBITO,LATITUD,LONGITUD,LAT_DECIMAL,LON_DECIMAL,ALTITUD,CVE_CARTA,POB_TOTAL,POB_MASCULINA,POB_FEMENINA,TOTAL DE VIVIENDAS HABITADAS
0,10010001,NaN,01,Aguascalientes,Ags.,001,Aguascalientes,1,Aguascalientes,U,"21°52´47.362N""","102°17´45.768W""",21.879822,-102.296046,1878,F13D19,863893,419168,444725,246259
1,10010094,NaN,01,Aguascalientes,Ags.,001,Aguascalientes,94,Granja Adelita,R,"21°52´18.749N""","102°22´24.710W""",21.871874,-102.373530,1901,F13D18,5,*,*,2
2,10010096,NaN,01,Aguascalientes,Ags.,001,Aguascalientes,96,Agua Azul,R,"21°53´01.522N""","102°21´25.639W""",21.883756,-102.357122,1861,F13D18,41,24,17,12
3,10010100,NaN,01,Aguascalientes,Ags.,001,Aguascalientes,100,Rancho Alegre,R,"21°51´16.556N""","102°22´21.884W""",21.854599,-102.372745,1879,F13D18,0,0,0,0
4,10010102,NaN,01,Aguascalientes,Ags.,001,Aguascalientes,102,Los Arbolitos [Rancho],R,"21°46´48.650N""","102°21´26.261W""",21.780180,-102.357295,1861,F13D18,8,*,*,2


In [6]:
# Eliminamos las columnas que no son de interés
COLUMNS_TO_DROP = ['MAPA', 'Estatus', 'NOM_ABR', 'AMBITO', 'LATITUD', 'LONGITUD',
                   'LAT_DECIMAL', 'LON_DECIMAL', 'ALTITUD', 'CVE_CARTA', 'POB_TOTAL',
                   'POB_MASCULINA', 'POB_FEMENINA', 'TOTAL DE VIVIENDAS HABITADAS']
dataset_inegi = dataset_inegi.drop(COLUMNS_TO_DROP, axis=1)

In [7]:
# Las claves de entidad y municipio serán tratadas numéricamente en la limpieza aunque posteriormente se les asignará el tipo de cadena de texto para tener el estándar.
dataset_inegi.dtypes

CVE_ENT    object
NOM_ENT    object
CVE_MUN    object
NOM_MUN    object
CVE_LOC     int64
NOM_LOC    object
dtype: object

In [8]:
# Revisamos la cantidad de filas y columnas del dataset
print("Shape of dataset_inegi: ", dataset_inegi.shape)

dataset_inegi_clean = dataset_inegi.drop_duplicates()
print("Shape of dataset_inegi_clean: ", dataset_inegi_clean.shape)

Shape of dataset_inegi:  (299568, 6)
Shape of dataset_inegi_clean:  (299568, 6)


In [9]:
# Revisamos las primeras filas del dataset con las columnas seleccionadas
dataset_inegi_clean.head()

,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,CVE_LOC,NOM_LOC
0,01,Aguascalientes,001,Aguascalientes,1,Aguascalientes
1,01,Aguascalientes,001,Aguascalientes,94,Granja Adelita
2,01,Aguascalientes,001,Aguascalientes,96,Agua Azul
3,01,Aguascalientes,001,Aguascalientes,100,Rancho Alegre
4,01,Aguascalientes,001,Aguascalientes,102,Los Arbolitos [Rancho]


In [10]:
# Revisamos las últimas filas del dataset con las columnas seleccionadas
dataset_inegi_clean.tail()

,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,CVE_LOC,NOM_LOC
299563,32,Zacatecas,058,Santa María de la Paz,39,San Isidro
299564,32,Zacatecas,058,Santa María de la Paz,40,San José
299565,32,Zacatecas,058,Santa María de la Paz,41,San Miguel Tepetitlán
299566,32,Zacatecas,058,Santa María de la Paz,42,San Rafael
299567,32,Zacatecas,058,Santa María de la Paz,43,Santa Fe


In [11]:
dataset_inegi_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299568 entries, 0 to 299567
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   CVE_ENT  299568 non-null  object
 1   NOM_ENT  299568 non-null  object
 2   CVE_MUN  299568 non-null  object
 3   NOM_MUN  299568 non-null  object
 4   CVE_LOC  299568 non-null  int64 
 5   NOM_LOC  299568 non-null  object
dtypes: int64(1), object(5)
memory usage: 13.7+ MB


In [12]:
print("Los valores únicos en cada columna son:\n", dataset_inegi_clean.nunique())

Los valores únicos en cada columna son:
 CVE_ENT        32
NOM_ENT        32
CVE_MUN       570
NOM_MUN      2332
CVE_LOC      5692
NOM_LOC    149004
dtype: int64


### 2.1.1 Estandarización de nombre de municipios

Con el fin de poder hacer un merge bajo los mismos nombres, hacemos una limpieza de los datos.

In [13]:
dataset_inegi_clean['NOM_ENT_Clean'] = dataset_inegi_clean['NOM_ENT'].apply(clean_text)
dataset_inegi_clean['NOM_MUN_Clean'] = dataset_inegi_clean['NOM_MUN'].apply(clean_text)
dataset_inegi_clean['NOM_LOC_Clean'] = dataset_inegi_clean['NOM_LOC'].apply(clean_text)

In [14]:
dataset_inegi_clean.shape

(299568, 9)

In [15]:
# Renombrar las columnas
dataset_inegi_clean.rename(columns={
    'CVE_ENT': 'CVE_ENT',
    'NOM_ENT': 'Entidad_inegi',
    'CVE_MUN': 'CVE_MUN',
    'NOM_MUN': 'Municipio_inegi',
    'CVE_LOC': 'CVE_LOC',
    'NOM_LOC': 'Localidad_inegi',
    'NOM_ENT_Clean': 'Entidad_c_inegi',
    'NOM_MUN_Clean': 'Municipio_c_inegi',
    'NOM_LOC_Clean': 'Localidad_c_inegi'
}, inplace=True)

#dataset_inegi['CVE_LOC'] = dataset_inegi['CVE_LOC'].astype(str)
# Las claves de entidad y municipio serán tratadas numéricamente en la limpieza aunque posteriormente se les asignará el tipo de cadena de texto para tener el estándar.
dataset_inegi_clean.dtypes

CVE_ENT              object
Entidad_inegi        object
CVE_MUN              object
Municipio_inegi      object
CVE_LOC               int64
Localidad_inegi      object
Entidad_c_inegi      object
Municipio_c_inegi    object
Localidad_c_inegi    object
dtype: object

In [16]:
dataset_inegi_clean.to_csv('../../data/dataset_inegi_clean.csv', index=False)